In [8]:
import tkinter as tk
import xlwings as xw
import pandas as pd
import numpy as np

In [1]:
# pyinstaller --windowed TD.py //Для того, чтобы сделать exe файл без консоли



path_to_KPI_file = '../../../KPI техотдела 2024.xlsx'
path_to_plan_time_file = '../../../../../../ForEverybody/DepProduction/Daily 2024_Нач_смен.xlsm'

def middle_of_day(d):
    return pd.to_datetime(f'{d.year}-{d.month}-{d.day} 12:00:00')

def end_of_day(d):
    d += pd.Timedelta(days=1)
    return pd.to_datetime(f'{d.year}-{d.month}-{d.day} 00:00:00')

def end_of_shift(d):
    return end_of_day(d) if (d >= middle_of_day(d)) else middle_of_day(d)
def get_plan_time(machine, shift, day, month, year):
    date = pd.to_datetime(f'{year}-{month}-{day}')
    try:
        return plan_time_dict[machine][
            (plan_time_dict[machine]["Date"] == date) & (plan_time_dict[machine]["Shift"] == shift)][
            "working time-stops for maintenance"].iloc[0]
    except IndexError:
        return np.nan

def clear():
    KPI_sheet.range('B132').expand('table').value = None
    KPI_sheet.range('H132').expand('table').value = None
    KPI_sheet.range('M132').expand('table').value = None
    KPI_sheet.range('D10:D17').value = None

TableDowntime = pd.read_excel(path_to_KPI_file, sheet_name='Downtime (dowload from CPMS)',
                                  usecols=['Станок', 'Код простоя', 'Месяц', 'Число', 'Начало простоя', 'Конец простоя',
                                           'простой в мин.', 'Проблема', 'Узел'])
KPI_book = xw.Book(path_to_KPI_file)
KPI_sheet = KPI_book.sheets["KPI"]

machine = {
    "M222  ( Asahi/Gopfert 2100 Msk )": "ASAHI",
    "M213  ( Martine 924 Msk )": "924",
    "M232  ( RDC Goepfert 1624 )": "RDCGoepfert",
    "M214  ( Martine 1232 Msk )": "1232",
    "M221  ( Bobst/Flexo 1600 Msk )": "BOBST",
    "M211  ( Miniline 616 Msk )": "616",
    "M100  ( BHS 2500 )": "Corrugator",
    "M241  ( Tanabe JD BoxR 1450 )": "Tanabe"
}

# Меняем формат месяца из Apr в 04 и т.д. Цифру месяца берём из столбца месяц
TableDowntime['Начало простоя'] = pd.to_datetime(TableDowntime['Начало простоя'])
TableDowntime['Конец простоя'] = pd.to_datetime(TableDowntime['Конец простоя'])  # pandas понимает формат Apr, Mar и т.д.

# Разделяем простои длинной в несколько дней на несколько записей или если простой затрагивает 2 дня
# Пример 16.06 - период с 7-00 16.06.24 до 7-00 17.06.24
# Временно смещаем время на 7 часов, чтобы корректно вычислять число и месяц в формате с 7 до 7
TableDowntime['Начало простоя'] = pd.to_datetime(TableDowntime['Начало простоя'], format='%d %m %Y %H:%M:%S') - pd.Timedelta(hours=7)
TableDowntime['Конец простоя'] = pd.to_datetime(TableDowntime['Конец простоя'], format='%d %m %Y %H:%M:%S') - pd.Timedelta(hours=7)

# Делим запись с длинным простоем на записи с более короткими простоями по сменам (в днях не учитываем график с 7 до 7, т.к. даты временно смещены на 7 часов)
for i in range(len(TableDowntime)):  # цикл не пробегает по созданным в цикле записям
    cur_i = i
    # Пока затрагивается новая смена отрезаем запись и проверяем новую запись в таблице
    while (end_of_shift(TableDowntime.loc[cur_i, 'Начало простоя']) != end_of_shift(
            TableDowntime.loc[cur_i, 'Конец простоя'])):
        new_end = end_of_shift(TableDowntime.loc[cur_i, 'Начало простоя'])
        new_series = TableDowntime.loc[cur_i].copy()
        new_series.loc['Начало простоя'] = new_end
        TableDowntime.loc[cur_i, 'Конец простоя'] = new_end
        cur_i = len(TableDowntime)  # задаём индекс новой записи (только что созданной)
        TableDowntime.loc[len(TableDowntime)] = new_series
# Для новых записей также определяем число и месяц
TableDowntime["Число"] = TableDowntime['Начало простоя'].dt.day
TableDowntime["Месяц"] = TableDowntime['Начало простоя'].dt.month
TableDowntime["Год"] = TableDowntime['Начало простоя'].dt.year
TableDowntime['Смена'] = 1 + (TableDowntime['Начало простоя'] >= TableDowntime['Начало простоя'].apply(
    middle_of_day))  # чтобы можно было сортировать по смене

# Смещаем обратно на 7 часов
TableDowntime['Начало простоя'] = pd.to_datetime(TableDowntime['Начало простоя']) + pd.Timedelta(hours=7)
TableDowntime['Конец простоя'] = pd.to_datetime(TableDowntime['Конец простоя']) + pd.Timedelta(hours=7)
TableDowntime["простой в мин."] = ((TableDowntime['Конец простоя'] - TableDowntime[
    'Начало простоя']).dt.total_seconds() / 60).round().astype(int)
TableDowntime["Станок"] = TableDowntime["Станок"].apply(lambda x: machine[x])
#Берём время работы for maintenance
plan_time_dict = pd.read_excel(path_to_plan_time_file,
    skiprows=[1,2,3,4],
    usecols = ['Date', 'Shift', 'working time-stops for maintenance'],
    sheet_name=['Corrugator', '616', '924', '1232', 'BOBST', 'Tanabe', 'ASAHI', 'RDCGoepfert'])
for sheet_name in plan_time_dict.keys():
    plan_time_dict[sheet_name] = plan_time_dict[sheet_name].dropna()
    #Изменяем 07-19, 19-07 на 1 и 2 смену
    plan_time_dict[sheet_name]["Shift"] = plan_time_dict[sheet_name]["Shift"].apply(lambda s : 1 if (s =="07-19") else 2)
# группировка по столбцам месяц, число и станок
grouped_Downtime = TableDowntime.groupby(['Станок', 'Смена', 'Месяц', 'Число', 'Год']).agg({'простой в мин.': 'sum'}).reset_index()
# сопоставляем время работы for maintenance с записями о застоях
grouped_Downtime["Время работы for maintenance"] = grouped_Downtime.apply(lambda row: get_plan_time(row['Станок'], row['Смена'], row['Число'], row['Месяц'], row['Год']), axis=1)
grouped_Downtime["TD"] = grouped_Downtime["простой в мин."] / grouped_Downtime["Время работы for maintenance"] / 60 * 100
grouped_Downtime["Дата"] = grouped_Downtime.apply(lambda row: pd.to_datetime(f'{row["Год"]}-{row["Месяц"]}-{row["Число"]}'), axis=1)
# Возвращаем смену в исходный формат
grouped_Downtime["Смена"] = grouped_Downtime["Смена"].apply(lambda n: "07-19" if (n == 1) else "19-07")
grouped_Downtime = grouped_Downtime.sort_values(by='Дата', ascending=False)

def grouped_by_shift():
    #Записываем таблицу в файл
    KPI_sheet.range('A132').value = grouped_Downtime[
        ["Станок", "Смена", "Дата", "TD"]]
    KPI_sheet.range('A133').expand('down').value = None  # чистим индексы
def Today():
    # Выводим сегодняшнюю статистику
    today = pd.to_datetime(KPI_sheet.range('C6').value) - pd.Timedelta("1 day")
    day_grouped_Downtime = grouped_Downtime.groupby(['Станок', 'Дата']).agg(
        {"Время работы for maintenance": 'sum', "простой в мин.": "sum"}).reset_index()

    machine_in_rus = {
        'Corrugator': 'Гофроагрегат',
        '616': 'Мартин 616',
        '924': 'Мартин 924',
        '1232': 'Мартин 1232',
        'BOBST': 'Бобст',
        'ASAHI': 'Асахи',
        'RDCGoepfert': 'RDC Goepfert',
        'Tanabe': 'Tanabe'
    }

    machine_in_rus_keys = list(machine_in_rus.keys())
    day_grouped_Downtime["TD"] = day_grouped_Downtime["простой в мин."] / (
                day_grouped_Downtime["Время работы for maintenance"] * 60) * 100
    today_TD = day_grouped_Downtime.loc[day_grouped_Downtime["Дата"] == today, ['Станок', 'TD']]
    today_TD.set_index('Станок', inplace=True)  # inplace - вместо возвращения нового датафрейма измениться текущий
    for machine_name in (
            set(machine_in_rus_keys) - set(today_TD.index)):  # Для машин по которым данных не было заполняем нулями
        today_TD.loc[machine_name] = 0
    sorted_today_TD = np.array(
        today_TD.loc[['Corrugator', '616', '924', '1232', 'BOBST', 'ASAHI', 'RDCGoepfert', 'Tanabe']][
            "TD"])  # сортировка прихотливым индексированием

    KPI_sheet.range('D10').value = sorted_today_TD.reshape(len(sorted_today_TD), 1)
def grouped_by_day():
    day_grouped_Downtime = grouped_Downtime.groupby(['Станок', 'Дата']).agg(
        {"Время работы for maintenance": 'sum', "простой в мин.": "sum"}).reset_index()
    day_grouped_Downtime["TD"] = day_grouped_Downtime["простой в мин."] / (
                day_grouped_Downtime["Время работы for maintenance"] * 60) * 100
    day_grouped_Downtime = day_grouped_Downtime.sort_values(by='Дата', ascending=False)
    KPI_sheet.range('G132').value = day_grouped_Downtime[["Станок", "Дата", "TD"]]
    KPI_sheet.range('G133').expand('down').value = None  # чистим индексы
def grouped_by_month():
    # По Месяцам
    month_grouped_Downtime = grouped_Downtime.groupby(['Станок', 'Год', 'Месяц']).agg(
        {"Время работы for maintenance": 'sum', "простой в мин.": "sum"}).reset_index()
    month_grouped_Downtime["TD"] = month_grouped_Downtime["простой в мин."] / (
                month_grouped_Downtime["Время работы for maintenance"] * 60) * 100

    KPI_sheet.range('L132').value = month_grouped_Downtime[["Станок", "Год", "Месяц", "TD"]]
    KPI_sheet.range('L133').expand('down').value = None  # чистим индексы

def TD_all():
    Today()
    grouped_by_shift()
    grouped_by_day()
    grouped_by_month()

TD_all()

c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\gorya\AppDa

In [1]:
"Daily 2024_Нач_смен" == "Daily_2024_Нач_смен"

False

In [9]:
def f():
    print("OK")
def f2(f, text):
    print(text)
    f()
f2(f, "function is started")

function is started
OK


In [14]:
df = pd.DataFrame({"a": [1,1,2], "b": [2,2,2]})
df.index = df["a"]
df.loc[1]

,a,b
a,,
1,1,2
1,1,2


In [30]:
def replace_rus_to_eng(s):
    rus = "уУеЕНхХаАрРоОсСТ"
    end = "yYeEHxXaApPoOcCT"
    for i in range(len(rus)):
        s = s.replace(rus[i], end[i])
    return s

True